### Zusammenfassung: 

-Script importiert Daten <br>
-Fasst Review-Text zusammen <br> 
-Generiert Automatische Antwort basierend auf Sentiment-Score <br>
-Zufällige Zeilen-Auswahl auf Knopfdruck <br>

In [4]:
# Utility 
import pandas as pd
import string
#from gensim.summarization import summarize
#import difflib
# import nltk
# nltk.download('punkt')

# Summarizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

# Widget und Display 
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output
print(widgets.Button.on_click.__doc__)
pd.set_option('display.max_colwidth', -1)

# github: https://github.com/Krisselack/FHWN_Analysewerkzeuge

Register a callback to execute when the button is clicked.

        The callback will be called with one argument, the clicked button
        widget instance.

        Parameters
        ----------
        remove: bool (optional)
            Set to true to remove the callback from the list of callbacks.
        


Autoren: 
Oliver Greimeister
Benjamin Chissico 
Christian Brandstätter

In [5]:
# Einlesen der Daten
df = pd.read_pickle("./subj_data.pkl")

In [6]:
df.reset_index()

index             dateAdded           dateUpdated  \
0     0      2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
1     1      2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
2     2      2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
3     3      2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
4     4      2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
...  ..                       ...                   ...   
4571  4995   2017-03-06T14:59:25Z  2017-09-04T11:19:31Z   
4572  4996   2017-03-06T14:59:25Z  2017-09-04T11:19:31Z   
4573  4997   2017-03-06T14:59:25Z  2017-09-04T11:19:31Z   
4574  4998   2017-03-06T14:59:25Z  2017-09-04T11:19:31Z   
4575  4999   2017-03-06T14:59:25Z  2017-09-04T11:19:31Z   

                                                                          name  \
0     Amazon Kindle E-Reader 6" Wifi (8th Generation, 2016)                      
1     Amazon Kindle E-Reader 6" Wifi (8th Generation, 2016)                      
2     Amazon Kindle E-Reader 6" Wifi (8th Generation, 2016)                      
3     Amazon Kindle E-Reader 6" Wifi (8th Generation, 2016)                      
4     Amazon Kindle E-Reader 6" Wifi (8th Generation, 2016)                      
...                                                     ...                      
4571  Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers   
4572  Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers   
4573  Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers   
4574  Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers   
4575  Fire Tablet with Alexa, 7" Display, 16 GB, Magenta - with Special Offers   

       brand  \
0     Amazon   
1     Amazon   
2     Amazon   
3     Amazon   
4     Amazon   
...      ...   
4571  Amazon   
4572  Amazon   
4573  Amazon   
4574  Amazon   
4575  Amazon   

                                                                                                                                                                                                                                                                                                         categories  \
0     Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessories,Used:Tablets,E-Readers,E-Readers & Accessories,Computers/Tablets & Networking,Used:Computers Accessories,iPads Tablets,All Tablets,Tablets & E-readers,Computers & Tablets,Amazon,Tablets & eBook Readers   
1     Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessories,Used:Tablets,E-Readers,E-Readers & Accessories,Computers/Tablets & Networking,Used:Computers Accessories,iPads Tablets,All Tablets,Tablets & E-readers,Computers & Tablets,Amazon,Tablets & eBook Readers   
2     Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessories,Used:Tablets,E-Readers,E-Readers & Accessories,Computers/Tablets & Networking,Used:Computers Accessories,iPads Tablets,All Tablets,Tablets & E-readers,Computers & Tablets,Amazon,Tablets & eBook Readers   
3     Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessories,Used:Tablets,E-Readers,E-Readers & Accessories,Computers/Tablets & Networking,Used:Computers Accessories,iPads Tablets,All Tablets,Tablets & E-readers,Computers & Tablets,Amazon,Tablets & eBook Readers   
4     Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessories,Used:Tablets,E-Readers,E-Readers & Accessories,Computers/Tablets & Networking,Used:Computers Accessories,iPads Tablets,All Tablets,Tablets & E-readers,Computers & Tablets,Amazon,Tablets & eBook Readers   
...                                                                                                                                                                                                                                                          

In [7]:
# NOT SO Successful Approach
# res = summarize(df["reviews.text"].str.cat)
# summarize(document1)

### Summarizer funktioniert so: 
Extractive Summarization: These methods rely on extracting several parts, such as phrases and sentences, from a piece of text and stack them together to create a summary. Therefore, identifying the right sentences for summarization is of utmost importance in an extractive method.

https://becominghuman.ai/text-summarization-in-5-steps-using-nltk-65b21e352b65

- Häufigkeit der Wörter
- Splitten der Sätze
- Addieren der Häufigkeiten pro Satz (durch Zeichenlänge) 
- Auswahl des Satzes mit dem höchsten Score

In [8]:
# Erstellt Häufigkeitstabelle
# word_tokenize: Worttrennung
# function annotation 

def _create_frequency_table(text_string) -> dict:

    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()

    freqTable = dict()
    for word in words:
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1
        
    return freqTable

In [9]:
# Wertigkeit der Sätze anhand der einzelnen Worthäufigkeiten 
# word_tokenize: Satztrennung

def _score_sentences(sentences, freqTable) -> dict:
    sentenceValue = dict()
    
    for sentence in sentences:
        word_count_in_sentence = (len(word_tokenize(sentence)))
        for wordValue in freqTable:
            if wordValue in sentence.lower():
                if sentence[:10] in sentenceValue:
                    sentenceValue[sentence[:10]] += freqTable[wordValue]
                else:
                    sentenceValue[sentence[:10]] = freqTable[wordValue]
               
        sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] // word_count_in_sentence
    
    return sentenceValue

In [10]:
def _find_average_score(sentenceValue) -> int:
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]
            # Average value of a sentence from original text
    average = int(sumValues / len(sentenceValue))

    return average

In [11]:
# Auswahl der Sätze über Durchschnitt!

def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        
        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] > (threshold):
            summary += " " + sentence
            sentence_count += 1
        
    return summary

In [12]:
# summarizing function

def _summarizer(text): 

    # 1 Create the word frequency table
    freq_table = _create_frequency_table(text)

    '''
    We already have a sentence tokenizer, so we just need 
    to run the sent_tokenize() method to create the array of sentences.
    '''

    # 2 Tokenize the sentences
    sentences = sent_tokenize(text)

    # 3 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(sentences, freq_table)

    # 4 Find the threshold
    threshold = _find_average_score(sentence_scores)

    # 5 Important Algorithm: Generate the summary
    summary = _generate_summary(sentences, sentence_scores, 1.5 * threshold)

    return(summary)

In [13]:
# Summarizer hat Probleme, wenn keine Punktuation am Ende steht! 
# Hier die Korrektur: 
df["reviews.text"] = df["reviews.text"].str.strip()
df["satzende"] = df["reviews.text"].str[-1:].isin(list(string.punctuation)).map({True: '', False: '.'})
df["reviews.satz"] = df[['reviews.text', 'satzende']].apply(lambda x: ''.join(x), axis=1)

In [14]:
df["ZF"] = df["reviews.satz"].map(_summarizer)

In [17]:
df["ZF_janein"] = df["ZF"].str.len()>0

### 3 Klassen: kleiner 0  (schlecht) größer 0.5 (gut) und dazwischen (mittel)


In [18]:
schlecht = "Your answer saddens us very deeply. Please contact us again."
mittel = "At least you bought our product."
gut = "Thank you very much for your good review!"

In [19]:
df["Answer_minimal"] = ""
df.loc[df["polarityCleanedNoStopWords"]<0, "Answer_minimal"] = schlecht
df.loc[df["polarityCleanedNoStopWords"]>0.5, "Answer_minimal"] = gut

In [20]:
df.loc[df["Answer_minimal"].str.len()<2, "Answer_minimal"] = mittel

## Formen der Auto-Antwort

In [21]:
# Wenn summary vorhanden, erstelle lange Antwort mit summary. 

df["Answer"] = ""
df.loc[df["ZF_janein"], "Answer"] =df[df["ZF_janein"]]["Answer_minimal"] + " You said:" + df[df["ZF_janein"]]["ZF"]

In [22]:
df.loc[df["Answer"].str.len()<3, "Answer"] = df.loc[df["Answer"].str.len()<3, "Answer_minimal"]

## Zufällige Datenauswahl mit Output: Auto-generierte Antwort

In [23]:
button = widgets.Button(description="Click Me!")
output = widgets.Output()
widget1 = widgets.Output()
display(button, output)

def on_button_clicked(b):
    
    with output:
        selection = df[["name", "polarityCleanedNoStopWords", "reviews.text", "ZF", "Answer"]].sample(n=1)
        selection.columns = ["Produkt", "Sentiment", "Review", "Summary", "Auto-Antwort" ]
        
        with widget1:
            clear_output()
            display(selection)

button.on_click(on_button_clicked)
hbox = widgets.HBox([widget1])
display(hbox)

Button(description='Click Me!', style=ButtonStyle())

Output()